In [1]:
import os
import glob
import time
import numpy as np
import pandas as pd
import astropy.io.fits as pf
import matplotlib.pyplot as plt
from skyfield.api import load, wgs84, EarthSatellite, S, W

In [2]:
# Load TLEs for all passages
with open('../test_data/3leComplete.txt') as f:
    all_tles = f.readlines()
    f.close()   

# Split TLE list into individual lists for each TLE
all_tles = [i.strip() for i in all_tles]
tles = [all_tles[x:x+3] for x in range(0, len(all_tles), 3)]

# Reduce TLEs to Starlink only
starlink_tles = []
for tle in tles:
    if "STARLINK" in tle[0]:
        starlink_tles.append(tle)

# Obtain satellite passages
passed_sats = pd.read_pickle('../test_data/passed_satellites_20221023LSC.p') #passages = passages_20221023LSC.p
keys = list(passed_sats)

# Find any Starlink TLEs in the passages
idx = []
starlinks = np.asarray(starlink_tles).flatten()
for key in keys:
    mascara_tle1 = passed_sats[key]['TLE line1'].strip()
    i = np.where(starlinks == mascara_tle1)[0] #this is not going to be fast for big lists...
    if i.size > 0:
        idx.append(i[0] - 1) #appending the name of the starlink sat
        
# Now have indices for the flattened Starlink TLE list --> divide by 3 to get indices for the original list
orig_idx = [int(x/3) for x in idx]
slk_mas_tles = res_list = [starlink_tles[i] for i in orig_idx]

# Remove 0 labeling of first line of TLE because that's the proper format
for tle in slk_mas_tles:
    tle[0] = tle[0][2:]

print(f'Number of satellites recorded for this day: {len(all_tles)}')
print(f'Number of them that were Starlinks: {len(starlink_tles)}')
print(f'Number of Starlinks that passed MASCARA: {len(slk_mas_tles)}')

Number of satellites recorded for this day: 72222
Number of them that were Starlinks: 3179
Number of Starlinks that passed MASCARA: 931


In [3]:
### DETERMINE TIMERANGE OF IMAGES

t0 = time.time()
ts = load.timescale()
folder = '../test_data/diff_images/'
images = glob.glob(f'{folder}diff_*.fits.gz')

dates = [] 
names = []

# Loop over each image
print('Determining timerange')
for img in images:
    _, header = pf.getdata(img, header=True)
    midJD = ts.tt_jd(header[12]).tt
    names.append(img)
    dates.append(midJD)

oldest = min(dates)
newest = max(dates)

# Converting to utc for convenience
newest = ts.tt_jd(newest).utc_strftime()
oldest = ts.tt_jd(oldest).utc_strftime()
print('Oldest date:', oldest)
print('Newest date:', newest)

beg = pd.to_datetime(oldest)
end = pd.to_datetime(newest)
rng = pd.date_range(beg, end, freq='0.05H').to_pydatetime().tolist() #every 3 minutes=
timerange = ts.from_datetimes(rng)
print(f'Runtime: {str(time.time() - t0)}')

Determining timerange
Oldest date: 2022-10-24 00:24:08 UTC
Newest date: 2022-10-24 08:43:22 UTC
Runtime: 4.108465909957886


In [4]:
timerange

<Time tt=[2459876.51756 ... 2459876.8633933333] len=167>

In [5]:
### DETERMINE SUNLIT PERIODS

t0 = time.time()

# Load the DE421 planetary ephemeris to get positions of Sun and Earth
eph = load('de421.bsp')
sun = eph['sun']
earth = eph['earth']

# Location of MASCARA
mascara = wgs84.latlon(latitude_degrees=29.26111*S, longitude_degrees=70.73139*W, elevation_m=2400)

# Set the observer to be MASCARA at ESO's La Silla Observatory
observer = earth + mascara

# Convert our TLEs to Skyfield EarthSatellites
print('Creating EarthSatellites')
sats = []
for tle in slk_mas_tles:
    sats.append(EarthSatellite(tle[1], tle[2], tle[0]))

# We now begin to create a list of timeranges for which a satellite is sunlit
starlink_times = {}

print('Finding sunlit times')
for sat in sats:
    # Check when satellite is sunlit 
    sunlit = sat.at(timerange).is_sunlit(eph)

    # Obtain the indices of the first and last TRUE element for each sequence of TRUE elements 
    x = 0
    idx = []
    for i, n in enumerate(sunlit):
        if n and x==0:
            idx.append(i)
            x=1
        if not n and x==1:
            idx.append(i-1)
            x=0
        if n and i==len(sunlit)-1:
            idx.append(i)

    # Obtain times corresponding to the inidces found 
    sunlit_ = [timerange.tt[i] for i in idx]

    # Now split into separate list --> now have the start and end time for each sunlit period
    values = [sunlit_[i:i + 2] for i in range(0, len(sunlit_), 2)]
    starlink_times[sat.name] = values

Creating EarthSatellites
Finding sunlit times


In [6]:
list(starlink_times)[0:10]

['STARLINK-2181',
 'STARLINK-2196',
 'STARLINK-2641',
 'STARLINK-2606',
 'STARLINK-3115',
 'STARLINK-3627',
 'STARLINK-3557',
 'STARLINK-1119',
 'STARLINK-1502',
 'STARLINK-2195']

In [7]:
### FINDING IF EACH SUNLIT STARLINK IS WITHIN THE IMAGE DATE

# We have a dictionary where each key is a Starlink ID. 
# In each key, we have a list of sunlit periods where each element is a list of the start and end time. 
# Must now loop through every Starlink ID and check if any of the sunlit periods are within the midJD of the image.


img_with_sunlit_starlink = []
print('Checking when each sunlit Starlink is within image')
t0 = time.time()

for i, name in enumerate(names):
    X = True

    for sat in list(starlink_times):
        for sunlitrng in starlink_times[sat]:
            if dates[i] >= sunlitrng[0] and dates[i] <= sunlitrng[1]:
                img_with_sunlit_starlink.append(name)
                X = False
                break

        if not X:
            break

print(f'# images expected to have Starlink trails visible: {len(img_with_sunlit_starlink)} out of {len(names)}')

Checking when each sunlit Starlink is within image
# images expected to have Starlink trails visible: 4 out of 5


### Check when satellite is above horizon:


In [8]:
ts = load.timescale()

mascara = wgs84.latlon(latitude_degrees=29.26111*S, longitude_degrees=70.73139*W, elevation_m=2400)
t0 = ts.utc(2022, 10, 24)
t1 = ts.utc(2022, 10, 25)

sat = sats[100]
t, events = sat.find_events(mascara, t0, t1, altitude_degrees=30)
for ti, event in zip(t, events):
    name = ('rise above 30°', 'culminate', 'set below 30°')[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)

2022 Oct 24 01:12:48 rise above 30°
2022 Oct 24 01:14:00 culminate
2022 Oct 24 01:15:13 set below 30°
2022 Oct 24 16:00:10 rise above 30°
2022 Oct 24 16:01:26 culminate
2022 Oct 24 16:02:44 set below 30°


In [9]:
from skyfield.almanac import find_discrete, risings_and_settings
from pytz import timezone

eph = load('de421.bsp')

f = risings_and_settings(eph, sun, mascara)
tz = timezone('Chile/Continental')

for t, updown in zip(*find_discrete(t0, t1, f)):
    print(t.astimezone(tz).strftime('%a %d %H:%M'),
          'rises' if updown else 'sets')

Mon 24 06:57 rises
Mon 24 19:57 sets


### TWILIGHT
Might not be any need to do this since MASCARA only opens when the right conditions are met

Could be useful for more in-depth analysis

In [10]:
import datetime as dt
from pytz import timezone
from skyfield import almanac
from skyfield.api import N, W, wgs84, load

# Figure out local midnight.
zone = timezone('Chile/Continental')
now = zone.localize(dt.datetime.now())
midnight = now.replace(hour=0, minute=0, second=0, microsecond=0)
next_midnight = midnight + dt.timedelta(days=1)

ts = load.timescale()
t0 = ts.utc(2022, 10, 24)
t1 = ts.utc(2022, 10, 26) #ts.from_datetime(end) 
eph = load('de421.bsp')

mascara = wgs84.latlon(latitude_degrees=29.26111*S, longitude_degrees=70.73139*W, elevation_m=2400)
f = almanac.dark_twilight_day(eph, mascara)
times, events = almanac.find_discrete(t0, t1, f)

previous_e = f(t0).item()
for t, e in zip(times, events):
    tstr = str(t.astimezone(zone))[:16]
    if previous_e < e:
        print(tstr, ' ', almanac.TWILIGHTS[e], 'starts')
    else:
        print(tstr, ' ', almanac.TWILIGHTS[previous_e], 'ends')
    previous_e = e

2022-10-23 21:21   Astronomical twilight ends
2022-10-24 05:32   Astronomical twilight starts
2022-10-24 06:02   Nautical twilight starts
2022-10-24 06:31   Civil twilight starts
2022-10-24 06:56   Day starts
2022-10-24 19:58   Day ends
2022-10-24 20:22   Civil twilight ends
2022-10-24 20:52   Nautical twilight ends
2022-10-24 21:22   Astronomical twilight ends
2022-10-25 05:31   Astronomical twilight starts
2022-10-25 06:01   Nautical twilight starts
2022-10-25 06:30   Civil twilight starts
2022-10-25 06:55   Day starts
2022-10-25 19:59   Day ends
2022-10-25 20:23   Civil twilight ends
2022-10-25 20:52   Nautical twilight ends


In [11]:
for t, e in zip(times, events):
    tstr = str(t.astimezone(zone))[:16]
    print(tstr, almanac.TWILIGHTS[e])

2022-10-23 21:21 Night
2022-10-24 05:32 Astronomical twilight
2022-10-24 06:02 Nautical twilight
2022-10-24 06:31 Civil twilight
2022-10-24 06:56 Day
2022-10-24 19:58 Civil twilight
2022-10-24 20:22 Nautical twilight
2022-10-24 20:52 Astronomical twilight
2022-10-24 21:22 Night
2022-10-25 05:31 Astronomical twilight
2022-10-25 06:01 Nautical twilight
2022-10-25 06:30 Civil twilight
2022-10-25 06:55 Day
2022-10-25 19:59 Civil twilight
2022-10-25 20:23 Nautical twilight
2022-10-25 20:52 Astronomical twilight


As you can see from the above code, if the new light level is brighter then we say that the new level “starts”, but if the new level is darker then we say the previous level “ends” — so instead of saying “astronomical twilight starts at 21:23” we say “nautical twilight ends at 21:23.” That’s why the code keeps up with previous_e and compares it to the new level of twilight.

0 = Dark of night <br>
1 = Astronomical twilight <br>
2 = Nautical twilight <br>
3 = Civil twilight <br>
4 = Daytime <br>

should focus on end of nautical twilight (start of astronomical twilight) $\rightarrow$ start of astronomical twilight


### Check if satellite elevation is above horizon

In [12]:
testsat = sats[0]

difference = testsat - mascara
topocentric = difference.at(t)
alt, az, distance = topocentric.altaz()

if alt.degrees > 0:
    print('The Starlink is above the horizon')

print('Altitude:', alt)
print('Azimuth:', az)
print('Distance: {:.1f} km'.format(distance.km))

Altitude: -33deg 11' 34.6"
Azimuth: 221deg 59' 45.2"
Distance: 7935.6 km


In [14]:
for t in timerange:
    topocentric = difference.at(t)
    alt, az, distance = topocentric.altaz()

    if alt.degrees > 0:
        print(f'{t.utc_strftime()}: The Starlink is above the horizon ({alt})')
        
#     if alt.degrees < 0:
#         print(f'{t.utc_strftime()}: The Starlink is below the horizon ({alt})')

2022-10-24 00:24:08 UTC: The Starlink is above the horizon (34deg 09' 11.8")
2022-10-24 00:27:08 UTC: The Starlink is above the horizon (40deg 28' 51.3")
2022-10-24 00:30:08 UTC: The Starlink is above the horizon (08deg 41' 44.5")
2022-10-24 02:03:08 UTC: The Starlink is above the horizon (03deg 18' 26.6")
2022-10-24 02:06:08 UTC: The Starlink is above the horizon (03deg 16' 14.3")


In [15]:
# Get range between when above to below

above = []
below = []
for t in timerange:
    topocentric = difference.at(t)
    alt, az, distance = topocentric.altaz()
    
    if alt.degrees > 0:
        above.append(t.utc_strftime())
        
    if alt.degrees < 0:
        below.append(t.utc_strftime())

list(zip(above, below))

[('2022-10-24 00:24:08 UTC', '2022-10-24 00:33:08 UTC'),
 ('2022-10-24 00:27:08 UTC', '2022-10-24 00:36:08 UTC'),
 ('2022-10-24 00:30:08 UTC', '2022-10-24 00:39:08 UTC'),
 ('2022-10-24 02:03:08 UTC', '2022-10-24 00:42:08 UTC'),
 ('2022-10-24 02:06:08 UTC', '2022-10-24 00:45:08 UTC')]

In [16]:
import datetime as dt
from skyfield.timelib import Time
(Time.utc_datetime(timerange[1]) - Time.utc_datetime(timerange[0])).seconds


180

In [17]:
t, events = sat.find_events(mascara, t0, t1, altitude_degrees=18)
below = t[2::3]
above = t[1::3]
periods = list(zip(below, above))
periods

[(<Time tt=2459876.5535628465>, <Time tt=2459876.5521877958>),
 (<Time tt=2459877.169904248>, <Time tt=2459877.168467647>),
 (<Time tt=2459877.504829305>, <Time tt=2459877.5038143196>),
 (<Time tt=2459878.18684299>, <Time tt=2459878.186169061>)]

In [18]:
img = 'diff_48506263LSC.fits.gz'
_, header = pf.getdata(f'../test_data/diff_images/{img}', header=True)
jd0 = header[7]
jd1 = header[8]
midJD = header[12]

In [19]:
srt = ts.tt_jd(2459876.517563686).utc_strftime()
fin = ts.tt_jd(2459879.517563686).utc_strftime()

beg = pd.to_datetime(srt)
end = pd.to_datetime(fin)
rng = pd.date_range(beg, end, freq='0.05H').to_pydatetime().tolist() #every 3 minutes

times = ts.from_datetimes(rng)
sunlit = sat.at(times).is_sunlit(eph)

Number of images expected to have Starlink trails visible: 32

['48508923', '48507687', '48506263', '48508093', '48510552', '48506374', '48509667', '48508038', '48507013', '48507007', '48508075', '48507951', '48510677', '48509481', '48507697', '48509572', '48506795', '48509870', '48510791', '48510736', '48510870', '48506731', '48509980', '48506341', '48506377', '48507667', '48507022', '48508889', '48510747', '48508951', '48508711', '48510668']

In [19]:
def calcSatSunPhase(sat, loc, ephemeris, tEv):
    earth = eph['earth']

    vecObserverSat = (sat - loc).at(tEv)
    vecSunSat = (earth + sat).at(tEv)
    phase = vecObserverSat.separation_from(vecSunSat)

    return phase

python full_code.py -n 3000 -f '20221023LSC' <br>
Number of satellites recorded for this day: 72222 <br>
Number of them that were Starlinks: 3179 <br>
Number of Starlinks that passed MASCARA: 931 <br>
Collecting images from 20221023LSC <br>
Determining timerange <br>
Oldest date: 2022-10-24 00:24:08 UTC <br>
Newest date: 2022-10-24 08:59:06 UTC <br>
Runtime: 3700.03729224205 <br>
Creating EarthSatellites <br>
Finding sunlit times <br>
Checking when each sunlit Starlink is within image <br>
Triple nested For Loop for 3000 images took 0.06397652626037598 <br>
Number of images expected to have Starlink trails visible: 2971 <br>
Elapsed:1:01:44.14,User=2928.225,System=601.843,CPU=95.3%.

## Alt, Az of Sun and Satellite + height of Satellite

In [33]:
def altaz_skyfield(satellite):
    
    import os
    from skyfield.api import load, wgs84, EarthSatellite
    
    # La Silla info
    confdir = '../fotos-python3/bringfiles'
    confdir = os.path.join(confdir, 'siteinfo.dat') 
    dtype = [('sitename', '|U20'), ('lat', 'float32'), ('lon', 'float32'), ('height', 'float32'), ('ID', '|U2')]
    siteinfo = np.genfromtxt(confdir, dtype=dtype)  
    camid = 'LSC'
    mask = siteinfo['ID'] == camid[:2]
    siteinfo = siteinfo[mask]
    lat = siteinfo[0][1]
    lon = siteinfo[0][2]
    ele = siteinfo[0][3]
    
    # Set date of observation
    ts = load.timescale()
    t = ts.utc(2022, 10, 23)
    
    # Load ephemeris
    eph = load('de421.bsp')
    earth = eph['earth']
    sun = eph['sun']
    
    # Find Sun alt and az
    obs = earth + wgs84.latlon(latitude_degrees=lat, longitude_degrees=lon, elevation_m=ele)
    astro = obs.at(t).observe(sun) #astrometric
    app = astro.apparent()
    sun_alt, sun_az, dist = app.altaz()
    
    # Now for Starlink
    sat = EarthSatellite(satellite[1], satellite[2], satellite[0], ts)
    
    # Check whether the sat is above or below the horizon from MASCARA, and in which direction to look for it
    mascara = wgs84.latlon(latitude_degrees=lat, longitude_degrees=lon, elevation_m=ele)
    diff = sat - mascara
    
    # Position of sat relative to MASCARA
    topocentric = diff.at(t)
    alt, az, dist = topocentric.altaz()

    if alt.degrees > 0:
        print(f'{satellite[0]} is above the horizon')
    else:
        print(f'{satellite[0]} is below the horizon')
        
    print('Sun Alt:', sun_alt)
    print('Sun Az:', sun_az)
    print('Sat Alt:', alt)
    print('Sat Az:', az)
    print('Sat Dist: {:.1f} km'.format(dist.km))
    
    # wWhere among the stars the sat will be positioned
    ra, dec, radec_dist = topocentric.radec()
    
    # Satellite elevation
    geocentric = sat.at(t)
    height = wgs84.height_of(geocentric)
    # subpoint = wgs84.subpoint(geocentric) # subpoint.elevation.km # Just another way of doing it!

    print(f'Sat elev above MASCARA: {height.km} km')
    print(f'Sat elev above Earth:   {height.km + ele/1000.} km')
    
    return ra, dec

ra, dec = altaz_skyfield(slk_mas_tles[44])

STARLINK-2599 is below the horizon
Sun Alt: -13deg 55' 28.8"
Sun Az: 248deg 14' 51.8"
Sat Alt: -72deg 52' 36.2"
Sat Az: 157deg 46' 21.6"
Sat Dist: 12770.0 km
Sat elev above MASCARA: 548.7416848407868 km
Sat elev above Earth:   551.1110847431305 km
